# Aufgabe 1 : Lese dir Seite 35 aus dem Buch Thermoformen in der Praxis durch und erstelle einen Funktion die die Dimensionen einer Kavität als Input nimmt und die praktisch erreichbare Wanddickenvverteilung ausgibt. Runde das Ergebnis auf eine Nachkommastelle.
# Angaben : Kavität LxBxH = 150x150x30 mm ; Grundfläche : LxB = 180x180 mm ; Ausgangsmaterialstärke : 2 mm 

In [9]:
def wall_thickness_distribution(base_length,base_width,cavity_length, cavity_width, cavity_heigth, initial_thickness):
    F1 = base_length * base_width
    F2 = F1 + 2*cavity_width*cavity_heigth + 2*cavity_length*cavity_heigth

    average_thickness = round(initial_thickness * (F1 / F2), 1)

    return average_thickness


In [10]:
wall_thickness_distribution(180,180,150,150,30,2)

1.3

# Aufgabe 2 : Lese dir Seite 502 und 457 des Buchs Roarks Formulas for Stress and Strain durch und erstelle eine Funktion die die Durchbiegung des Trays berechnet. Als Input nimm die Angaben der Grundfläche und der erreichten Wanddickenvereilung als Materialstärke aus Aufgabe 1 und gehe davon aus das ein Gewicht von 1 kg uniform auf die Platte einwirkt. Gehe von der Materialart PS-HI aus was ein E-Modul von : 2150 N/mm2 hat.

In [122]:
from scipy.interpolate import interp1d


def get_alpha_func():
    alpha = [0.0444, 0.0616, 0.0770, 0.0906, 0.1017, 0.1110, 0.1335, 0.1400, 0.1417]
    ratio = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 3.0, 4.0, 5.0]

    f = interp1d(ratio, alpha)

    return f

def get_alpha(ratio):

    alpha_func = get_alpha_func()

    if ratio <= 5:
        alpha = alpha_func(ratio)
    else:
        alpha = 0.1421

    return alpha

def get_force_per_unit_area(weight, base_length, base_width):
    gravity_constant=9.81
    force = weight * gravity_constant
    force_per_unit_area = force / (base_length * base_width)
    return force_per_unit_area



#W = total applied load (force)
#q = load per unit area : q = W / (base_length * base_width) : N/mm2
#y = vertical deflection of plate (length);
#E = modulus of elasticity (force per unit area);
#t = plate thickness
#alpha = a constant that depends on the aspect ratio of the plate
# y = -alpha * q/(a*b) * (b^4) / (E * (t^3))

def deflection(base_length, base_width, thickness, materialtyp, weight):
    modulus = {"PS-HI": 2150}

    elastic_modulus = modulus[materialtyp]

    dims = [base_length, base_width]
    dims.sort()

    ratio = dims[1] / dims[0]

    alpha = get_alpha(ratio)
    force_per_unit_area = get_force_per_unit_area(weight, base_length, base_width)

    deflection = (-alpha* force_per_unit_area * (dims[0]**4))/(elastic_modulus*(thickness**3))

    return deflection




    


    


In [165]:
deflection(180, 180, 1.3, "PS-HI",1)

-2.9876414137671876

# Aufgabe 3 : Verbinde die Aufgabenstellung der vorherigen beiden Aufgaben und erstelle eine Funktion die ausgehend von einer erwünschten maximalen Durchbiegung des Trays die Ausgangsmaterialstärke berechnet
# Berechne die Ausgangsmaterialstärke für eine maximale Durchbiegung von 1 mm bei gleichen Bedingungen wie in Aufgabe 1 und 2 

In [170]:
def initial_material_thickness(deflection, base_length, base_width, cavity_length, cavity_width, cavity_heigth, materialtyp, weight):

    F1 = base_length * base_width
    F2 = F1 + 2*cavity_width*cavity_heigth + 2*cavity_length*cavity_heigth

    thickness_ratio = F2 / F1

    modulus = {"PS-HI": 2150}

    elastic_modulus = modulus[materialtyp]

    dims = [base_length, base_width]
    dims.sort()

    ratio = dims[1] / dims[0]

    alpha = get_alpha(ratio)
    force_per_unit_area = get_force_per_unit_area(weight, base_length, base_width)

    thickness = ((-alpha * force_per_unit_area * (dims[0]**4)) / (elastic_modulus * deflection))**(1/3)*thickness_ratio

    return thickness

In [166]:
# Überprüfung der Funktionen
initial_material_thickness(-2.9876414137671876, 180, 180, 150, 150, 30, "PS-HI", 1)

2.022222222222222

In [171]:
initial_material_thickness(-1, 180, 180, 150, 150, 30, "PS-HI", 1)

2.9125386829690068